# Notebook to issue corrections for missed donations from institutions with "Southern" and "Brirish" in name

### Such as "Columbia Southern University" and "University of Brirish [sic] Columbia"

## Import and clean data

In [18]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DIR = os.getcwd()
print(DIR)



df = pd.read_csv(DIR + "/2019to2020contributions.csv",
                    parse_dates = ["contribution_receipt_date"]) 

df.columns

/Users/lukedobrovic/Data/Spectator/personal-campaign-contributions


/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (12,18,20,32,50,51,53,54,57,59) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['committee_id', 'committee_name', 'report_year', 'report_type',
       'image_number', 'line_number', 'transaction_id', 'file_number',
       'committee_name.1', 'entity_type', 'entity_type_desc',
       'unused_contbr_id', 'contributor_prefix', 'contributor_name',
       'recipient_committee_type', 'recipient_committee_org_type',
       'recipient_committee_designation', 'contributor_first_name',
       'contributor_middle_name', 'contributor_last_name',
       'contributor_suffix', 'contributor_street_1', 'contributor_street_2',
       'contributor_city', 'contributor_state', 'contributor_zip',
       'contributor_employer', 'contributor_occupation', 'contributor_id',
       'receipt_type', 'receipt_type_desc', 'receipt_type_full', 'memo_code',
       'memo_code_full', 'contribution_receipt_date',
       'contribution_receipt_amount', 'contributor_aggregate_ytd',
       'candidate_id', 'candidate_name', 'candidate_first_name',
       'candidate_last_name', 'candidate_middle_na

In [19]:
df_selections = df[[ "committee_name", "contribution_receipt_date",
                    "contribution_receipt_amount", "contributor_employer", 
                   "contributor_occupation", "contributor_first_name", 
                   "contributor_middle_name", "contributor_last_name"]]

faux = ["MISSOURI", "BRITISH", "DISTRICT", "CHICAGO", "INTERNATIONAL", "MO", "UNIVERSITY HOSPITAL", "FLORIDA",
       "CAROLINA"]

df_selections = df_selections[~df_selections.contributor_employer.str.contains('|'.join(faux))]

df_selections = df_selections[df_selections["contribution_receipt_amount"] >= 0]  

## Make dataframe with contributions from "Southern" and "Brirish" affiliates

In [20]:
errors = ["SOUTHERN", "BRIRISH"]

df_errors = df_selections[df_selections["contributor_employer"].str.contains('|'.join(errors))]
print(df_errors.shape)
print("Total errors donations: ", df_errors["contribution_receipt_amount"].sum())

(262, 8)
Total errors donations:  24500.300000000003


## Make Biden and Trump dataframes with contributions from missed institutions, compute donation totals

In [21]:
df_biden = df_errors[df_errors["committee_name"].str.contains("BIDEN FOR PRESIDENT")]
df_biden.reset_index()
print("biden shape: ", df_biden.shape)

df_trump = df_errors[df_errors["committee_name"].str.contains("DONALD J. TRUMP FOR PRESIDENT, INC.")]
df_trump.reset_index()
print("trump shape: ", df_trump.shape)

biden shape:  (3, 8)
trump shape:  (15, 8)


In [22]:
print("biden total $: ", df_biden["contribution_receipt_amount"].sum())
print("trump total $: ", df_trump["contribution_receipt_amount"].sum())

biden total $:  150.0
trump total $:  1528.75


## Track donations to committes from error institutions

In [23]:
df_committees = df_southern.groupby(["committee_name"], as_index=False).sum()
df_committees.sort_values(by="contribution_receipt_amount", ascending=False, inplace=True)
df_committees.reset_index()

,index,committee_name,contribution_receipt_amount
0,3,"BYRNE FOR SENATE, INC.",7000.00
1,8,JERRY CARL FOR CONGRESS,3150.00
2,6,FRIENDS OF SESSIONS SENATE COMMITTEE INC,2500.00
3,0,ACTBLUE,2179.55
4,7,HIGHTOWER FOR ALABAMA,2000.00
5,9,MIKE ROGERS FOR CONGRESS,1668.00
6,5,"DONALD J. TRUMP FOR PRESIDENT, INC.",1528.75
7,13,TRUMP MAKE AMERICA GREAT AGAIN COMMITTEE,1125.00
8,10,REPUBLICAN NATIONAL COMMITTEE,1000.00
9,4,DEFEND AMERICA PAC,834.00


## Find occupations of donations from error institutions 

In [24]:
df_occ = df_southern.drop_duplicates(subset=["contributor_last_name", "contributor_middle_name",
                                              "contributor_middle_name"])

occupations = df_occ["contributor_occupation"].value_counts()

occupations

PROFESSOR                10
ACADEMIC ADVISOR          3
EDITOR                    2
WEB DESIGNER              1
VETERANS OUTREACH         1
INSTRUCTOR                1
ADMINISTRATOR             1
ENGLISH PROFESSOR         1
IEA SPECIALIST            1
MEDIA MANAGER             1
FINANCIAL AID ADVISOR     1
REFERENCE LIBRARIAN       1
UNIVERSITY PROFESSOR      1
TEACHER                   1
PRESIDENT                 1
HIGHER ED                 1
FINANCIAL AID             1
ADMISSIONS COUNSELOR      1
COURSE TECHNOLOGIST       1
COMMS MANAGER             1
Name: contributor_occupation, dtype: int64